In [1]:
from google.colab import drive
import requests
from IPython.display import Video, display

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os
import numpy as np

# Set paths
landmark_folder = "/content/drive/MyDrive/ASL_project/Dataset/landmark_vectors"
FEATURE_SIZE = 375  # Expected number of landmark features

# Function to ensure all CSV files have exactly 375 columns
def fix_csv_shapes(folder):
    for root, dirs, files in os.walk(folder):
        for filename in files:
            if filename.endswith(".csv"):
                file_path = os.path.join(root, filename)

                try:
                    df = pd.read_csv(file_path, header=None)
                    current_columns = df.shape[1]

                    # Convert to NumPy array to avoid Pandas alignment issues
                    data = df.to_numpy()

                    # Fix CSV files with missing columns
                    if current_columns < FEATURE_SIZE:
                        print(f"Fixing {file_path}: {current_columns} → {FEATURE_SIZE} columns (padding).")
                        missing_cols = FEATURE_SIZE - current_columns
                        padding = np.zeros((data.shape[0], missing_cols))  # Ensure zero-padding
                        data = np.hstack((data, padding))  # Stack along columns

                    # Fix CSV files with too many columns
                    elif current_columns > FEATURE_SIZE:
                        print(f"Fixing {file_path}: {current_columns} → {FEATURE_SIZE} columns (trimming).")
                        data = data[:, :FEATURE_SIZE]  # Trim extra columns

                    # Overwrite file with fixed column count
                    pd.DataFrame(data).to_csv(file_path, header=False, index=False)

                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

# Run the function to fix all CSV files
fix_csv_shapes(landmark_folder)

print("✅ All CSV files are now correctly padded to 375 columns!")


Error processing /content/drive/MyDrive/ASL_project/Dataset/landmark_vectors/study/55368_study.csv: No columns to parse from file
Error processing /content/drive/MyDrive/ASL_project/Dataset/landmark_vectors/study/55369_study.csv: No columns to parse from file
Error processing /content/drive/MyDrive/ASL_project/Dataset/landmark_vectors/science/49637_science.csv: No columns to parse from file
✅ All CSV files are now correctly padded to 375 columns!


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder

# Set paths
train_path = "/content/drive/MyDrive/ASL_project/Dataset/landmark_xy"
test_path = "/content/drive/MyDrive/ASL_project/Dataset/landmark_xy_test"
allowed_labels = {'book', 'drink', 'computer_bk'}  # Only use these classes

# Set fixed values
MAX_FRAMES = 30  # Fixed sequence length
FEATURE_SIZE = 150  # Fixed feature size
LEARNING_RATE = 0.0001  # Custom learning rate

# Function to load sequences and labels
def load_sequences(folder):
    sequences = []
    labels = []
    class_names = sorted([name for name in os.listdir(folder) if name in allowed_labels])  # Filter classes
    print(f"Class names: {class_names}")

    label_encoder = LabelEncoder()
    label_encoder.fit(class_names)  # Encode labels

    for class_name in class_names:
        class_path = os.path.join(folder, class_name)

        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                if filename.endswith(".csv"):
                    file_path = os.path.join(class_path, filename)

                    try:
                        df = pd.read_csv(file_path, header=None)

                        # Handle cases where the CSV has fewer or more than FEATURE_SIZE columns
                        if df.shape[1] < FEATURE_SIZE:
                            print(f"Warning: {file_path} has {df.shape[1]} columns. Padding to {FEATURE_SIZE}.")
                            missing_cols = FEATURE_SIZE - df.shape[1]
                            padding = np.zeros((df.shape[0], missing_cols))
                            df = pd.concat([df, pd.DataFrame(padding)], axis=1)

                        elif df.shape[1] > FEATURE_SIZE:
                            print(f"Warning: {file_path} has {df.shape[1]} columns. Trimming to {FEATURE_SIZE}.")
                            df = df.iloc[:, :FEATURE_SIZE]

                        total_frames = df.shape[0]  # Number of frames in the video

                        # Select 30 frames evenly spaced
                        step = max(1, total_frames // MAX_FRAMES)  # Ensure at least step 1
                        selected_frames = df.iloc[::step].values[:MAX_FRAMES]  # Take frames in jumps

                        # If we got fewer than 30 frames, pad with zeros
                        if selected_frames.shape[0] < MAX_FRAMES:
                            padding = np.zeros((MAX_FRAMES - selected_frames.shape[0], FEATURE_SIZE))
                            selected_frames = np.vstack([selected_frames, padding])

                        sequences.append(selected_frames)
                        labels.append(label_encoder.transform([class_name])[0])

                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")
                        continue

    X = np.array(sequences)
    y = np.array(labels)

    print(f"First X: {X[0, 0, :]}") if len(X) > 0 else print("No data found.")
    print(f"Final dataset shape: X={X.shape}, y={y.shape}")

    return X, y, label_encoder

# Load train and test datasets
X_train, y_train, label_encoder = load_sequences(train_path)
X_test, y_test, _ = load_sequences(test_path)  # Use the same label encoder

# Check if dataset is loaded properly
if X_train.shape[0] == 0 or X_test.shape[0] == 0:
    raise ValueError("No valid training or testing data found. Please check dataset format.")

# Build LSTM Model
model = Sequential([
    Masking(mask_value=0.0, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(128, return_sequences=True),
    Dropout(0.1),
    LSTM(64),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Set custom learning rate
optimizer = Adam(learning_rate=LEARNING_RATE)

# Compile model with custom learning rate
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=2)

# Save model
model.save("/content/drive/MyDrive/ASL_project/Models/asl_lstm_fixed.h5")

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

Class names: ['book', 'computer_bk', 'drink']
First X: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.53914732 0.26693732 0.55658275 0.22724473 0.56657374 0.22963184
 0.57625431 0.23325479 0.52460027 0.22467184 0.51484

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.4595 - loss: 1.0398 - val_accuracy: 0.5776 - val_loss: 0.8323
Epoch 2/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 17s 44ms/step - accuracy: 0.6349 - loss: 0.8157 - val_accuracy: 0.6552 - val_loss: 0.7734
Epoch 3/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 53ms/step - accuracy: 0.7622 - loss: 0.6713 - val_accuracy: 0.6638 - val_loss: 0.7319
Epoch 4/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - accuracy: 0.7426 - loss: 0.6111 - val_accuracy: 0.6810 - val_loss: 0.7273
Epoch 5/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 54ms/step - accuracy: 0.7510 - loss: 0.6207 - val_accuracy: 0.7328 - val_loss: 0.6190
Epoch 6/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 19s 48ms/step - accuracy: 0.7860 - loss: 0.5252 - val_accuracy: 0.7414 - val_loss: 0.6483
Epoch 7/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 22s 55ms/step - accuracy: 0.7993 - loss: 0.4977 - val_accuracy: 0.7586 - val_loss: 0.6269
Epoch 8/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 44ms/step - accuracy: 0.8509 - loss: 0.4238 - val_accurac

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7784 - loss: 0.5325
Test Accuracy: 0.76


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score

# Set paths
train_path = "/content/drive/MyDrive/ASL_project/Dataset/landmark_xy"
test_path = "/content/drive/MyDrive/ASL_project/Dataset/landmark_xy_test"
allowed_labels = {'book', 'drink', 'computer_bk'}  # Only use these classes

# Set fixed values
MAX_FRAMES = 30  # Fixed sequence length
FEATURE_SIZE = 150  # Fixed feature size
LEARNING_RATE = 0.0001  # Custom learning rate

# Function to load sequences and labels
def load_sequences(folder):
    sequences = []
    labels = []
    class_names = sorted([name for name in os.listdir(folder) if name in allowed_labels])  # Filter classes
    print(f"Class names: {class_names}")

    label_encoder = LabelEncoder()
    label_encoder.fit(class_names)  # Encode labels

    for class_name in class_names:
        class_path = os.path.join(folder, class_name)

        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                if filename.endswith(".csv"):
                    file_path = os.path.join(class_path, filename)

                    try:
                        df = pd.read_csv(file_path, header=None)

                        # Handle cases where the CSV has fewer or more than FEATURE_SIZE columns
                        if df.shape[1] < FEATURE_SIZE:
                            print(f"Warning: {file_path} has {df.shape[1]} columns. Padding to {FEATURE_SIZE}.")
                            missing_cols = FEATURE_SIZE - df.shape[1]
                            padding = np.zeros((df.shape[0], missing_cols))
                            df = pd.concat([df, pd.DataFrame(padding)], axis=1)

                        elif df.shape[1] > FEATURE_SIZE:
                            print(f"Warning: {file_path} has {df.shape[1]} columns. Trimming to {FEATURE_SIZE}.")
                            df = df.iloc[:, :FEATURE_SIZE]

                        total_frames = df.shape[0]  # Number of frames in the video

                        # Select 30 frames evenly spaced
                        step = max(1, total_frames // MAX_FRAMES)  # Ensure at least step 1
                        selected_frames = df.iloc[::step].values[:MAX_FRAMES]  # Take frames in jumps

                        # If we got fewer than 30 frames, pad with zeros
                        if selected_frames.shape[0] < MAX_FRAMES:
                            padding = np.zeros((MAX_FRAMES - selected_frames.shape[0], FEATURE_SIZE))
                            selected_frames = np.vstack([selected_frames, padding])

                        sequences.append(selected_frames)
                        labels.append(label_encoder.transform([class_name])[0])

                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")
                        continue

    X = np.array(sequences)
    y = np.array(labels)

    print(f"First X: {X[0, 0, :]}") if len(X) > 0 else print("No data found.")
    print(f"Final dataset shape: X={X.shape}, y={y.shape}")

    return X, y, label_encoder

# Load train and test datasets
X_train, y_train, label_encoder = load_sequences(train_path)
X_test, y_test, _ = load_sequences(test_path)  # Use the same label encoder

# Check if dataset is loaded properly
if X_train.shape[0] == 0 or X_test.shape[0] == 0:
    raise ValueError("No valid training or testing data found. Please check dataset format.")

# Build LSTM Model
model = Sequential([
    Masking(mask_value=0.0, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(128, return_sequences=True),
    Dropout(0.1),
    LSTM(64),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Set custom learning rate
optimizer = Adam(learning_rate=LEARNING_RATE)

# Compile model with custom learning rate
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Custom callback to compute precision & recall per epoch
class PrecisionRecallCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        y_pred_train = np.argmax(self.model.predict(X_train), axis=1)
        y_pred_test = np.argmax(self.model.predict(X_test), axis=1)

        train_precision = precision_score(y_train, y_pred_train, average='weighted', zero_division=0)
        train_recall = recall_score(y_train, y_pred_train, average='weighted', zero_division=0)

        test_precision = precision_score(y_test, y_pred_test, average='weighted', zero_division=0)
        test_recall = recall_score(y_test, y_pred_test, average='weighted', zero_division=0)

        print(f"\n📌 Epoch {epoch+1}: Train Precision={train_precision:.4f}, Train Recall={train_recall:.4f}, "
              f"Test Precision={test_precision:.4f}, Test Recall={test_recall:.4f}")

# Train model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=2, callbacks=[PrecisionRecallCallback()])

# Save model
model.save("/content/drive/MyDrive/ASL_project/Models/asl_lstm_fixed.h5")

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")


Class names: ['book', 'computer_bk', 'drink']
First X: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.53914732 0.26693732 0.55658275 0.22724473 0.56657374 0.22963184
 0.57625431 0.23325479 0.52460027 0.22467184 0.51484

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

📌 Epoch 1: Train Precision=0.6930, Train Recall=0.6480, Test Precision=0.6357, Test Recall=0.5690
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 68ms/step - accuracy: 0.4190 - loss: 1.0692 - val_accuracy: 0.5690 - val_loss: 0.8504
Epoch 2/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

📌 Epoch 2: Train Precision=0.7367, Train Recall=0.7249, Test Precision=0.6481, Test Recall=0.6034
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.6874 - loss: 0.8046 - val_accuracy: 0.6034 - val_loss: 0.9158
Epoch 3/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step

📌 Epoch 3: Train Precision=0.7676, Train Recall=0.7552, Test Precision=0.6018, Test Recall=0.6034
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 48ms/step - accuracy: 0.7216 - loss: 0.6998 - val_accuracy: 0.6034 - val_loss: 0.8788
Epoch 4/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.8553 - loss: 0.4370
Test Accuracy: 0.84


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report

# Set paths
train_path = "/content/drive/MyDrive/ASL_project/Dataset/landmark_xy"
test_path = "/content/drive/MyDrive/ASL_project/Dataset/landmark_xy_test"
allowed_labels = {'book', 'drink', 'computer_bk'}  # Only use these classes

# Set fixed values
MAX_FRAMES = 30  # Fixed sequence length
FEATURE_SIZE = 150  # Fixed feature size
LEARNING_RATE = 0.0001  # Custom learning rate

# Function to load sequences and labels
def load_sequences(folder):
    sequences = []
    labels = []
    class_names = sorted([name for name in os.listdir(folder) if name in allowed_labels])  # Filter classes
    print(f"Class names: {class_names}")

    label_encoder = LabelEncoder()
    label_encoder.fit(class_names)  # Encode labels

    for class_name in class_names:
        class_path = os.path.join(folder, class_name)

        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                if filename.endswith(".csv"):
                    file_path = os.path.join(class_path, filename)

                    try:
                        df = pd.read_csv(file_path, header=None)

                        # Handle cases where the CSV has fewer or more than FEATURE_SIZE columns
                        if df.shape[1] < FEATURE_SIZE:
                            print(f"Warning: {file_path} has {df.shape[1]} columns. Padding to {FEATURE_SIZE}.")
                            missing_cols = FEATURE_SIZE - df.shape[1]
                            padding = np.zeros((df.shape[0], missing_cols))
                            df = pd.concat([df, pd.DataFrame(padding)], axis=1)

                        elif df.shape[1] > FEATURE_SIZE:
                            print(f"Warning: {file_path} has {df.shape[1]} columns. Trimming to {FEATURE_SIZE}.")
                            df = df.iloc[:, :FEATURE_SIZE]

                        total_frames = df.shape[0]  # Number of frames in the video

                        # Select 30 frames evenly spaced
                        step = max(1, total_frames // MAX_FRAMES)  # Ensure at least step 1
                        selected_frames = df.iloc[::step].values[:MAX_FRAMES]  # Take frames in jumps

                        # If we got fewer than 30 frames, pad with zeros
                        if selected_frames.shape[0] < MAX_FRAMES:
                            padding = np.zeros((MAX_FRAMES - selected_frames.shape[0], FEATURE_SIZE))
                            selected_frames = np.vstack([selected_frames, padding])

                        sequences.append(selected_frames)
                        labels.append(label_encoder.transform([class_name])[0])

                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")
                        continue

    X = np.array(sequences)
    y = np.array(labels)

    print(f"First X: {X[0, 0, :]}") if len(X) > 0 else print("No data found.")
    print(f"Final dataset shape: X={X.shape}, y={y.shape}")

    return X, y, label_encoder

# Load train and test datasets
X_train, y_train, label_encoder = load_sequences(train_path)
X_test, y_test, _ = load_sequences(test_path)  # Use the same label encoder

# Check if dataset is loaded properly
if X_train.shape[0] == 0 or X_test.shape[0] == 0:
    raise ValueError("No valid training or testing data found. Please check dataset format.")

# Build LSTM Model
model = Sequential([
    Masking(mask_value=0.0, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(128, return_sequences=True),
    Dropout(0.1),
    LSTM(64),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Set custom learning rate
optimizer = Adam(learning_rate=LEARNING_RATE)

# Compile model with custom learning rate
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Track the best epoch for confusion matrix
best_epoch = 0
best_val_accuracy = 0.0
best_y_pred = None

# Custom callback to compute precision, recall & track best epoch
class PrecisionRecallCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        global best_epoch, best_val_accuracy, best_y_pred

        y_pred_train = np.argmax(self.model.predict(X_train), axis=1)
        y_pred_test = np.argmax(self.model.predict(X_test), axis=1)

        train_precision = precision_score(y_train, y_pred_train, average='weighted', zero_division=0)
        train_recall = recall_score(y_train, y_pred_train, average='weighted', zero_division=0)

        test_precision = precision_score(y_test, y_pred_test, average='weighted', zero_division=0)
        test_recall = recall_score(y_test, y_pred_test, average='weighted', zero_division=0)
        val_accuracy = logs["val_accuracy"]

        print(f"\n📌 Epoch {epoch+1}: Train Precision={train_precision:.4f}, Train Recall={train_recall:.4f}, "
              f"Test Precision={test_precision:.4f}, Test Recall={test_recall:.4f}")

        # Track best epoch
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_epoch = epoch + 1
            best_y_pred = y_pred_test.copy()

# Train model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=2, callbacks=[PrecisionRecallCallback()])

# Save model
model.save("/content/drive/MyDrive/ASL_project/Models/asl_lstm_fixed.h5")

# Evaluate best epoch with confusion matrix
print(f"\n✅ Best Epoch: {best_epoch}, Best Validation Accuracy: {best_val_accuracy:.4f}")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, best_y_pred)
report = classification_report(y_test, best_y_pred, target_names=label_encoder.classes_, digits=4)

print("\n🔹 Confusion Matrix:\n", conf_matrix)
print("\n🔹 Classification Report:\n", report)

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")


Class names: ['book', 'computer_bk', 'drink']
First X: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.53914732 0.26693732 0.55658275 0.22724473 0.56657374 0.22963184
 0.57625431 0.23325479 0.52460027 0.22467184 0.51484

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step

📌 Epoch 1: Train Precision=0.7268, Train Recall=0.6830, Test Precision=0.5646, Test Recall=0.5776
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - accuracy: 0.5094 - loss: 1.0196 - val_accuracy: 0.5776 - val_loss: 0.8248
Epoch 2/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

📌 Epoch 2: Train Precision=0.7618, Train Recall=0.7622, Test Precision=0.6524, Test Recall=0.6552
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - accuracy: 0.7158 - loss: 0.7745 - val_accuracy: 0.6552 - val_loss: 0.7459
Epoch 3/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

📌 Epoch 3: Train Precision=0.7845, Train Recall=0.7832, Test Precision=0.5913, Test Recall=0.5948
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 57ms/step - accuracy: 0.7628 - loss: 0.6400 - val_accuracy: 0.5948 - val_loss: 0.7672
Epoch 4/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step




✅ Best Epoch: 24, Best Validation Accuracy: 0.8448

🔹 Confusion Matrix:
 [[33  0  1]
 [ 9 35  2]
 [ 5  1 30]]

🔹 Classification Report:
               precision    recall  f1-score   support

        book     0.7021    0.9706    0.8148        34
 computer_bk     0.9722    0.7609    0.8537        46
       drink     0.9091    0.8333    0.8696        36

    accuracy                         0.8448       116
   macro avg     0.8611    0.8549    0.8460       116
weighted avg     0.8735    0.8448    0.8472       116

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.8349 - loss: 0.5330
Test Accuracy: 0.82
